In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import requests
import random


In [2]:
ipo_age = pd.read_excel("C:/Users/restr/Documents/Springboard/Capstone project 2/IPO Project/IPO-age.xlsx")
company_ipo = pd.read_csv("C:/Users/restr/Documents/Springboard/Capstone project 2/IPO Project/company_ipo.csv")

In [3]:
# Print out the first few rows of each dataframe to check it's loaded correctly
print("IPO Age Data:")
print(ipo_age.head())

IPO Age Data:
   Offer date             IPO name Ticker     CUSIP Rollup   VC Dual  \
0         NaN                  NaN    NaN       NaN    NaN  NaN  NaN   
1  19750130.0              ROYSTER    NaN  78088610      .    0    0   
2  19750609.0                VARCO    VRC  92212610      .    0    0   
3  19750610.0         COORS ADOLPH    TAP  21701610      .    0    1   
4  19750715.0  KEYSTONE FOODS CORP    NaN  49348410      .    0    0   

  PostIssueShares Internet CRSP permanent ID  Founding  Unnamed: 11  \
0             NaN      NaN               NaN       NaN          NaN   
1               .        0             67898    1901.0          NaN   
2               .        0             63044    1908.0          NaN   
3        35418915        0             59248    1901.0          NaN   
4               .        0             61989    1965.0          NaN   

                                         Unnamed: 12 Unnamed: 13  
0          N=13,945 U.S. IPOs from Jan 1975-Dec 2022       

In [4]:
print("\nCompany IPO Data:")
print(company_ipo.head())


Company IPO Data:
   ID      IPO Date Symbol                    Company Name IPO Price Current  \
0   1  Dec 21, 2021   NVAC           NorthView Acquisition    $10.00  $10.03   
1   2  Dec 21, 2021   LSPR     Larkspur Health Acquisition    $10.00   $9.98   
2   3  Dec 21, 2021   SAGA            Sagaliam Acquisition    $10.00   $9.97   
3   4  Dec 17, 2021   RCAC  Revelstone Capital Acquisition    $10.00   $9.90   
4   5  Dec 17, 2021   APCA                  AP Acquisition    $10.00  $10.11   

   Return  
0   0.30%  
1  -0.20%  
2  -0.30%  
3  -1.00%  
4   1.10%  


In [5]:
# Data types
print(ipo_age.dtypes)

Offer date           float64
IPO name              object
Ticker                object
CUSIP                 object
Rollup                object
VC                    object
Dual                  object
PostIssueShares       object
Internet              object
CRSP permanent ID     object
Founding             float64
Unnamed: 11          float64
Unnamed: 12           object
Unnamed: 13           object
dtype: object


In [6]:
print(company_ipo.dtypes)

ID               int64
IPO Date        object
Symbol          object
Company Name    object
IPO Price       object
Current         object
Return          object
dtype: object


In [7]:
# Summary statistics
print(ipo_age.describe())

         Offer date      Founding  Unnamed: 11
count  1.394500e+04  13945.000000          0.0
mean   1.997482e+07   1652.747293          NaN
std    1.113588e+05    759.743699          NaN
min    1.975013e+07    -99.000000          NaN
25%    1.989033e+07   1956.000000          NaN
50%    1.996020e+07   1983.000000          NaN
75%    2.004112e+07   1994.000000          NaN
max    2.022123e+07   2021.000000          NaN


In [8]:
print(company_ipo.describe())

               ID
count  1765.00000
mean    883.00000
std     509.65593
min       1.00000
25%     442.00000
50%     883.00000
75%    1324.00000
max    1765.00000


In [9]:
# Counts of unique values
print(ipo_age['Ticker'].value_counts())

Ticker
TEMPA    5
SHOE     4
SNOW     4
OPEN     4
DERM     4
        ..
CEGE     1
DPS      1
PTB      1
SHMN     1
COYA     1
Name: count, Length: 12343, dtype: int64


### Removing unnecessary rows and columns: 
Above, we saw that the first row of the DataFrame is empty and should be removed. The last three columns ('Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13') are mostly filled with NaN values and instruction of the excel spreadsheet, which can also be removed. 

In [10]:
# Drop the first row
ipo_age = ipo_age.iloc[1:]

# Drop the last three columns
ipo_age = ipo_age.drop(columns=['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'])

print(ipo_age.head())

   Offer date             IPO name Ticker     CUSIP Rollup VC Dual  \
1  19750130.0              ROYSTER    NaN  78088610      .  0    0   
2  19750609.0                VARCO    VRC  92212610      .  0    0   
3  19750610.0         COORS ADOLPH    TAP  21701610      .  0    1   
4  19750715.0  KEYSTONE FOODS CORP    NaN  49348410      .  0    0   
5  19750826.0          C. F. BRAUN    NaN  10564710      .  0    0   

  PostIssueShares Internet CRSP permanent ID  Founding  
1               .        0             67898    1901.0  
2               .        0             63044    1908.0  
3        35418915        0             59248    1901.0  
4               .        0             61989    1965.0  
5               .        0             58579    1909.0  


### Converting data types: 
Above we saw that 'Offer date' and 'Founding' columns are in a float format, even though they should be dates. We'll need to convert these columns.

In [11]:
# Convert 'Offer date' and 'Founding' to datetime format
ipo_age['Offer date'] = pd.to_datetime(ipo_age['Offer date'], format='%Y%m%d')
ipo_age['Founding'] = pd.to_datetime(ipo_age['Founding'], format='%Y', errors = 'coerce')

print(ipo_age.head())

  Offer date             IPO name Ticker     CUSIP Rollup VC Dual  \
1 1975-01-30              ROYSTER    NaN  78088610      .  0    0   
2 1975-06-09                VARCO    VRC  92212610      .  0    0   
3 1975-06-10         COORS ADOLPH    TAP  21701610      .  0    1   
4 1975-07-15  KEYSTONE FOODS CORP    NaN  49348410      .  0    0   
5 1975-08-26          C. F. BRAUN    NaN  10564710      .  0    0   

  PostIssueShares Internet CRSP permanent ID   Founding  
1               .        0             67898 1901-01-01  
2               .        0             63044 1908-01-01  
3        35418915        0             59248 1901-01-01  
4               .        0             61989 1965-01-01  
5               .        0             58579 1909-01-01  


In 'company_ipo' we need the 'IPO Date' column in datetime format.

In [12]:
# Convert 'IPO Date' to datetime format
company_ipo['IPO Date'] = pd.to_datetime(company_ipo['IPO Date'])

### Handling missing values 'ipo_age': 
The 'Ticker' column has a lot of missing values (NaN). In this case, we will drop these rows since the ticker is a unique identifier for a company.

In [13]:
# Drop rows where 'Ticker' is missing
ipo_age = ipo_age.dropna(subset=['Ticker'])

From the summary statistics, it seems like the 'Founding' column contains a value of '-99', which likely indicates missing data. We'll replace with NaN.

In [14]:
# Replace '-99' with 'NaN' in 'Founding' column
ipo_age['Founding'] = ipo_age['Founding'].replace('-99', pd.NA)

The VC (Venture Capital) Column: this column uses '.' and '9' to represent missing values. We'll replace these with NaN, and keep the valid values 0, 1, and 2.

In [15]:
ipo_age['VC'] = pd.to_numeric(ipo_age['VC'].replace(['.', '9', 9], np.nan), errors='coerce')

Dual Column: The 'Dual' column uses '.' and '9' to represent missing values as well. We'll handle this in the same way as we did for the 'VC' column.

In [16]:
ipo_age['Dual'] = pd.to_numeric(ipo_age['Dual'].replace(['.', '9', 9], np.nan), errors='coerce')

PostIssueShares Column: This column uses '-9' and '.' to represent missing values. In addition, this column also contains some '0' values. For now we will assume '0' is a valid number.

In [17]:
ipo_age['PostIssueShares'] = pd.to_numeric(ipo_age['PostIssueShares'].replace(['.', '-9', -9], np.nan), errors='coerce')

Internet Column: This column uses '.' to represent missing values, and '0' presumably represents non-internet-based companies. We'll replace '.' with NaN and keep '0'.

In [18]:
ipo_age['Internet'] = pd.to_numeric(ipo_age['Internet'].replace(['.', '9', 9], np.nan), errors='coerce')

In [19]:
print(ipo_age.head())

   Offer date                IPO name Ticker     CUSIP Rollup   VC  Dual  \
2  1975-06-09                   VARCO    VRC  92212610      .  0.0   0.0   
3  1975-06-10            COORS ADOLPH    TAP  21701610      .  0.0   1.0   
8  1975-10-21  KERR GLASS MANUFACTURI    KGM  49237610      .  0.0   0.0   
9  1975-10-28              LIQUID AIR   LANA  53631510      .  0.0   0.0   
10 1975-10-30   OCEANEERING INTERNATL    OII  67523210      .  NaN   NaN   

    PostIssueShares  Internet CRSP permanent ID   Founding  
2               NaN       0.0             63044 1908-01-01  
3        35418915.0       0.0             59248 1901-01-01  
8               NaN       0.0             58616 1903-01-01  
9               NaN       0.0             49825 1965-01-01  
10              NaN       0.0             58975 1965-01-01  


In [20]:
ipo_age.isnull().sum()

Offer date               0
IPO name                 0
Ticker                   0
CUSIP                   65
Rollup                   0
VC                      27
Dual                    99
PostIssueShares      11884
Internet                29
CRSP permanent ID        0
Founding              1869
dtype: int64

In [21]:
print(ipo_age.duplicated().sum())

0


In [22]:
print(ipo_age['VC'].unique())
print(ipo_age['Dual'].unique())
print(ipo_age['PostIssueShares'].unique())
print(ipo_age['Internet'].unique())

[ 0. nan  1.  2.]
[ 0.  1. nan  2.]
[           nan 3.54189150e+07 1.95238100e+06 ... 7.95761905e+08
 3.80000000e+07 3.91219000e+08]
[ 0.  1. nan]


In [23]:
print(ipo_age.describe())

                          Offer date            VC          Dual  \
count                          13549  13522.000000  13450.000000   
mean   1998-03-27 04:03:16.619676800      0.337746      0.119554   
min              1975-06-09 00:00:00      0.000000      0.000000   
25%              1989-12-14 00:00:00      0.000000      0.000000   
50%              1996-04-09 00:00:00      0.000000      0.000000   
75%              2005-03-28 00:00:00      1.000000      0.000000   
max              2022-12-29 00:00:00      2.000000      2.000000   
std                              NaN      0.541763      0.324909   

       PostIssueShares      Internet                       Founding  
count     1.665000e+03  13520.000000                          11680  
mean      1.646568e+08      0.064867  1981-12-26 01:55:46.027397248  
min       0.000000e+00      0.000000            1783-01-01 00:00:00  
25%       1.098293e+07      0.000000            1976-01-01 00:00:00  
50%       3.291620e+07      0.000000 

### Filtering data: 
Since we only have data of the offer price for IPOs from 2019-2020, we should filter the DataFrame accordingly.

In [24]:
# Filter for IPOs from 2019-2020
ipo_age = ipo_age[(ipo_age['Offer date'].dt.year >= 2019) & (ipo_age['Offer date'].dt.year <= 2020)]

print(ipo_age.head())

      Offer date                   IPO name Ticker      CUSIP Rollup   VC  \
13044 2019-01-04       China Sxt Pharms Inc   SXTC  G2161P108      0  0.0   
13045 2019-01-08                   Mdjm Ltd   MDJH  G59290109      0  0.0   
13046 2019-01-09  1895 Bancorp of Wisconsin   BCOW  28252W105      .  0.0   
13047 2019-01-09                  Mmtec Inc    MTC   G6181K10      0  0.0   
13048 2019-01-17          Rhinebeck Bancorp   RBKB  762093102      .  0.0   

       Dual  PostIssueShares  Internet CRSP permanent ID   Founding  
13044   0.0              NaN       0.0             18372 2006-01-01  
13045   0.0              NaN       0.0             18364 2014-01-01  
13046   0.0              NaN       0.0             18356 1995-01-01  
13047   0.0              NaN       0.0             18365 1956-01-01  
13048   0.0              NaN       0.0             18369 1960-01-01  


### Handling missing values 'company_ipo': 
Converting currency to numeric: The 'IPO Price' and 'Current' columns are currently strings because of the $ sign. Remove the dollar sign and convert the columns to a numeric format.

In [25]:
# Remove dollar sign and convert 'IPO Price' and 'Current' to numeric
company_ipo['IPO Price'] = pd.to_numeric(company_ipo['IPO Price'].str.replace('$', ''))
company_ipo['Current'] = pd.to_numeric(company_ipo['Current'].str.replace('$', ''))

In [26]:
print(company_ipo.head())

   ID   IPO Date Symbol                    Company Name  IPO Price  Current  \
0   1 2021-12-21   NVAC           NorthView Acquisition       10.0    10.03   
1   2 2021-12-21   LSPR     Larkspur Health Acquisition       10.0     9.98   
2   3 2021-12-21   SAGA            Sagaliam Acquisition       10.0     9.97   
3   4 2021-12-17   RCAC  Revelstone Capital Acquisition       10.0     9.90   
4   5 2021-12-17   APCA                  AP Acquisition       10.0    10.11   

   Return  
0   0.30%  
1  -0.20%  
2  -0.30%  
3  -1.00%  
4   1.10%  


Converting percentage to numeric: The 'Return' column should also be converted to a numeric format by removing the '%' symbol and dividing by 100.

In [27]:
# Remove '%' and convert 'Return' to numeric
company_ipo['Return'] = pd.to_numeric(company_ipo['Return'].str.replace('%', ''))/100

In [28]:
print(company_ipo.head())

   ID   IPO Date Symbol                    Company Name  IPO Price  Current  \
0   1 2021-12-21   NVAC           NorthView Acquisition       10.0    10.03   
1   2 2021-12-21   LSPR     Larkspur Health Acquisition       10.0     9.98   
2   3 2021-12-21   SAGA            Sagaliam Acquisition       10.0     9.97   
3   4 2021-12-17   RCAC  Revelstone Capital Acquisition       10.0     9.90   
4   5 2021-12-17   APCA                  AP Acquisition       10.0    10.11   

   Return  
0   0.003  
1  -0.002  
2  -0.003  
3  -0.010  
4   0.011  


We will rename 'Symbol' to 'Ticker' to be consistent with the ipo_age dataframe, so it will be easy to merge on this column.

In [29]:
# Rename 'Symbol' to 'Ticker'
company_ipo = company_ipo.rename(columns={'Symbol': 'Ticker'})

In [30]:
print(company_ipo.head())

   ID   IPO Date Ticker                    Company Name  IPO Price  Current  \
0   1 2021-12-21   NVAC           NorthView Acquisition       10.0    10.03   
1   2 2021-12-21   LSPR     Larkspur Health Acquisition       10.0     9.98   
2   3 2021-12-21   SAGA            Sagaliam Acquisition       10.0     9.97   
3   4 2021-12-17   RCAC  Revelstone Capital Acquisition       10.0     9.90   
4   5 2021-12-17   APCA                  AP Acquisition       10.0    10.11   

   Return  
0   0.003  
1  -0.002  
2  -0.003  
3  -0.010  
4   0.011  


Looking at the unique values in the 'Ticker' and 'IPO Date'/'Offer date' columns from both dataframes is a good way to understand the data before merging. We will also on thier counts using nunique()

In [31]:
print(company_ipo['Ticker'].unique())
print(company_ipo['IPO Date'].unique())
print(ipo_age['Ticker'].unique())
print(ipo_age['Offer date'].unique())

['NVAC' 'LSPR' 'SAGA' ... 'MDJH' 'MTC' 'SXTC']
<DatetimeArray>
['2021-12-21 00:00:00', '2021-12-17 00:00:00', '2021-12-16 00:00:00',
 '2021-12-15 00:00:00', '2021-12-14 00:00:00', '2021-12-13 00:00:00',
 '2021-12-10 00:00:00', '2021-12-09 00:00:00', '2021-12-08 00:00:00',
 '2021-12-07 00:00:00',
 ...
 '2019-02-12 00:00:00', '2019-02-08 00:00:00', '2019-02-07 00:00:00',
 '2019-02-04 00:00:00', '2019-02-01 00:00:00', '2019-01-31 00:00:00',
 '2019-01-17 00:00:00', '2019-01-09 00:00:00', '2019-01-08 00:00:00',
 '2019-01-04 00:00:00']
Length: 437, dtype: datetime64[ns]
['SXTC' 'MDJH' 'BCOW' 'MTC' 'RBKB' 'NRGX' 'NFE' 'INMB' 'ALEC' 'GOSS'
 'HARP' 'IMAC' 'AVDR' 'TCRR' 'HOTH' 'SOLY' 'MITO' 'SLGG' 'KLDO' 'SWAV'
 'FHL' 'TIGR' 'LEVI' 'TEAF' 'DTIL' 'WTRE' 'LYFT' 'PUYI' 'PBTS' 'RUHN'
 'NGM' 'SILK' 'TW' 'GHSI' 'PD' 'TUFN' 'PLMR' 'TPTX' 'MNRL' 'GNLN' 'HOOK'
 'PINS' 'ZM' 'WAFU' 'BYND' 'SY' 'TMDX' 'ATIF' 'RRBI' 'SCPL' 'YJ' 'TRVI'
 'PSN' 'AXLA' 'CRTX' 'HHR' 'MEC' 'MIST' 'NXTC' 'SPFI' 'JFIN' 'SONM' 'UBER'

In [32]:
print(company_ipo['Ticker'].nunique())
print(company_ipo['IPO Date'].nunique())
print(ipo_age['Ticker'].nunique())
print(ipo_age['Offer date'].nunique())


1758
437
386
183


In [33]:
matches = company_ipo['Ticker'].isin(ipo_age['Ticker'])
print(matches.sum())


371


The company_ipo dataframe has 1758 unique tickers and 437 unique dates. On the other hand, the ipo_age dataframe has 386 unique tickers and 183 unique dates.

This discrepancy may result in an imperfect merge, as not all tickers and dates from both dataframes may align.

Since since the ipo_age DataFrame has most of the variables we are going to investigate, we will import the IPO Price from company_ipo for the Tickers that we have in the ipo_age DataFrame. 

There are 371 matches out of 386 IPOs in the ipo_age Dataframe. We will investigate more about these cases after the merge. 

### Merging the DataFrames
We will create a new DataFrame merged_df which include all rows from `ipo_age`using left merge, importing the IPO Price column and the IPO Date column which we will use to check the integrity of the data.  
We use the 'Ticker' column on both DataFrames on= 'Ticker' for the merge.  

In [34]:
merged_df = ipo_age.merge(company_ipo[['Ticker', 'IPO Price', 'IPO Date']], on='Ticker', how='left')

print(merged_df.head())

  Offer date                   IPO name Ticker      CUSIP Rollup   VC  Dual  \
0 2019-01-04       China Sxt Pharms Inc   SXTC  G2161P108      0  0.0   0.0   
1 2019-01-08                   Mdjm Ltd   MDJH  G59290109      0  0.0   0.0   
2 2019-01-09  1895 Bancorp of Wisconsin   BCOW  28252W105      .  0.0   0.0   
3 2019-01-09                  Mmtec Inc    MTC   G6181K10      0  0.0   0.0   
4 2019-01-17          Rhinebeck Bancorp   RBKB  762093102      .  0.0   0.0   

   PostIssueShares  Internet CRSP permanent ID   Founding  IPO Price  \
0              NaN       0.0             18372 2006-01-01        4.0   
1              NaN       0.0             18364 2014-01-01        5.0   
2              NaN       0.0             18356 1995-01-01       10.0   
3              NaN       0.0             18365 1956-01-01        4.0   
4              NaN       0.0             18369 1960-01-01       10.0   

    IPO Date  
0 2019-01-04  
1 2019-01-08  
2 2019-01-09  
3 2019-01-08  
4 2019-01-17  


The 'IPO Date' from the company_ipo dataframe matches the 'Offer date' from the ipo_age dataframe for 369 of the records in the merged dataframe.

In [35]:
merged_df['date_match'] = merged_df['IPO Date'] == merged_df['Offer date']
print(merged_df['date_match'].value_counts())

date_match
True     369
False     17
Name: count, dtype: int64


We will now look into these 17 records to identify the cause of the mismatch.

In [36]:
mismatched_records = merged_df[merged_df['date_match'] == False]
print(mismatched_records)


    Offer date                        IPO name Ticker      CUSIP Rollup   VC  \
3   2019-01-09                       Mmtec Inc    MTC   G6181K10      0  0.0   
5   2019-01-30  Pimco Energy & Tactical Credit   NRGX   69346N10      0  0.0   
15  2019-02-15                         Soliton   SOLY  xxxxxxxxx      .  0.0   
20  2019-03-08               Futu Holdings Ltd    FHL  36118L106      .  1.0   
23  2019-03-27       Tortoise Essential Assets   TEAF  89148A103      0  0.0   
57  2019-05-09   Milestone Pharmaceuticals Inc   MIST  59935V107      0  1.0   
67  2019-05-17               Luckin Coffee Inc     LK  54951L109      0  2.0   
71  2019-06-06                 Gsx Techedu Inc    GSX  36257Y109      .  0.0   
84  2019-06-26   BlackRock Science and Tech II   BSTZ  09260K101      0  0.0   
113 2019-08-15                          9F Inc    JFG  65442R109      .  0.0   
142 2019-10-29   AllianzGI Artificial Int Fund    AIO   01883M10      0  0.0   
159 2019-11-26   Calamos Long/Short Equi

After doing some online research, the correct Offer Date for MTC is 2019-01-08 and for SOLY is 2019-02-19.
We will update them manually:

In [37]:
merged_df.loc[(merged_df['Ticker'] == 'MTC'), 'Offer date'] = pd.Timestamp('2019-01-08')
merged_df.loc[(merged_df['Ticker'] == 'SOLY'), 'Offer date'] = pd.Timestamp('2019-02-19')


Now we can drop the remaining mismatched records by the 'date_match' column, since none of these companies have the IPO offer price available. 

In [38]:
merged_df['date_match'] = merged_df['IPO Date'] == merged_df['Offer date']

In [39]:
merged_df = merged_df[merged_df['date_match'] == True]

In [40]:
# Check if we have any other missing values in IPO Price column. 

print(merged_df['IPO Price'].isnull().sum())


0


We don´t have any more missing values in the 'IPO Price' column :)

We will drop the columns we don´t need:

In [41]:
merged_df = merged_df.drop(columns=['IPO Date', 'date_match'])

In [42]:
print(merged_df.head())

  Offer date                   IPO name Ticker      CUSIP Rollup   VC  Dual  \
0 2019-01-04       China Sxt Pharms Inc   SXTC  G2161P108      0  0.0   0.0   
1 2019-01-08                   Mdjm Ltd   MDJH  G59290109      0  0.0   0.0   
2 2019-01-09  1895 Bancorp of Wisconsin   BCOW  28252W105      .  0.0   0.0   
3 2019-01-08                  Mmtec Inc    MTC   G6181K10      0  0.0   0.0   
4 2019-01-17          Rhinebeck Bancorp   RBKB  762093102      .  0.0   0.0   

   PostIssueShares  Internet CRSP permanent ID   Founding  IPO Price  
0              NaN       0.0             18372 2006-01-01        4.0  
1              NaN       0.0             18364 2014-01-01        5.0  
2              NaN       0.0             18356 1995-01-01       10.0  
3              NaN       0.0             18365 1956-01-01        4.0  
4              NaN       0.0             18369 1960-01-01       10.0  


We will create a new feature, 'Age at IPO', by subtracting the 'Founding' date from the 'Offer date'. This will provide useful information for our analysis.

In [43]:
# Create 'Company Age at IPO' column
merged_df['Age at IPO'] = (merged_df['Offer date'].dt.year - merged_df['Founding'].dt.year)


In [44]:
print(merged_df.head())

  Offer date                   IPO name Ticker      CUSIP Rollup   VC  Dual  \
0 2019-01-04       China Sxt Pharms Inc   SXTC  G2161P108      0  0.0   0.0   
1 2019-01-08                   Mdjm Ltd   MDJH  G59290109      0  0.0   0.0   
2 2019-01-09  1895 Bancorp of Wisconsin   BCOW  28252W105      .  0.0   0.0   
3 2019-01-08                  Mmtec Inc    MTC   G6181K10      0  0.0   0.0   
4 2019-01-17          Rhinebeck Bancorp   RBKB  762093102      .  0.0   0.0   

   PostIssueShares  Internet CRSP permanent ID   Founding  IPO Price  \
0              NaN       0.0             18372 2006-01-01        4.0   
1              NaN       0.0             18364 2014-01-01        5.0   
2              NaN       0.0             18356 1995-01-01       10.0   
3              NaN       0.0             18365 1956-01-01        4.0   
4              NaN       0.0             18369 1960-01-01       10.0   

   Age at IPO  
0          13  
1           5  
2          24  
3          63  
4          5

## Import Stock Price data from Yahoo for our IPO Ticker symbols

In [45]:
tickers = merged_df['Ticker'].unique().tolist()

We will include Nasdaq 100 ticker ^NDX which we will use later to calculate above-average returns 

In [46]:
tickers.append('^NDX')

In [47]:
print(tickers)

['SXTC', 'MDJH', 'BCOW', 'MTC', 'RBKB', 'NFE', 'INMB', 'ALEC', 'GOSS', 'HARP', 'IMAC', 'AVDR', 'TCRR', 'HOTH', 'SOLY', 'MITO', 'SLGG', 'KLDO', 'SWAV', 'TIGR', 'LEVI', 'DTIL', 'WTRE', 'LYFT', 'PUYI', 'PBTS', 'RUHN', 'NGM', 'SILK', 'TW', 'GHSI', 'PD', 'TUFN', 'PLMR', 'TPTX', 'MNRL', 'GNLN', 'HOOK', 'PINS', 'ZM', 'WAFU', 'BYND', 'SY', 'TMDX', 'ATIF', 'RRBI', 'SCPL', 'YJ', 'TRVI', 'PSN', 'AXLA', 'CRTX', 'HHR', 'MEC', 'NXTC', 'SPFI', 'JFIN', 'SONM', 'UBER', 'APLT', 'PSTL', 'AVTR', 'FSLY', 'BCYC', 'IDYA', 'RTLR', 'RVLV', 'CRWD', 'MWK', 'FVRR', 'CHWY', 'STOK', 'AKRO', 'BCEL', 'GO', 'PSNL', 'PRVL', 'WORK', 'CMBM', 'ADPT', 'BBIO', 'CHNG', 'MORF', 'KRTX', 'REAL', 'DOYU', 'AMK', 'FULC', 'MIRM', 'ORCC', 'PHR', 'AFYA', 'MDLA', 'EIC', 'CSTL', 'HCAT', 'LVGO', 'PROS', 'NOVA', 'BHAT', 'WSG', 'DT', 'KRUS', 'SNDL', 'HKIB', 'INMD', 'CFB', 'TXG', 'SDC', 'NET', 'STSA', 'SWTX', 'NVST', 'IGMS', 'DDOG', 'XGN', 'PING', 'OPRT', 'PTON', 'APRE', 'FREQ', 'VIE', 'BNTX', 'HBT', 'VIR', 'BRBR', 'BRP', 'AIH', 'CABA', 'H

To handle errors that might occur when trying to download the data we used: `try and except Exception as e` 

If an error occurs, the code inside the except block will be executed. The {e} part of the string will be replaced with the actual error message.


In [48]:
try:
    price_data = yf.download(tickers, start="2019-01-01", end="2023-12-30")
except Exception as e:
    print(f"Error downloading data: {e}")

[*********************100%***********************]  372 of 372 completed

46 Failed downloads:
- MDLA: No timezone found, symbol may be delisted
- HKIB: No timezone found, symbol may be delisted
- MOHO: No timezone found, symbol may be delisted
- DNK: No timezone found, symbol may be delisted
- CHNG: No timezone found, symbol may be delisted
- TPTX: No timezone found, symbol may be delisted
- WORK: No timezone found, symbol may be delisted
- CSPR: No timezone found, symbol may be delisted
- AYLA: No timezone found, symbol may be delisted
- MKD: No timezone found, symbol may be delisted
- SBTX: No timezone found, symbol may be delisted
- PPD: No timezone found, symbol may be delisted
- GBS: No timezone found, symbol may be delisted
- RUHN: No timezone found, symbol may be delisted
- IMAC: No timezone found, symbol may be delisted
- MCFE: No timezone found, symbol may be delisted
- PROS: No timezone found, symbol may be delisted
- VIE: No timezone found, symbol may be delisted
- PRVL: No

In [49]:
price_data.index = pd.to_datetime(price_data.index)

In [50]:
print(price_data.head())

           Adj Close                                             ... Volume  \
                ABCL ABNB ACCD ACI ADCT ADPT ADTX AFIB AFYA  AI  ...   YAYO   
Date                                                             ...          
2019-01-02       NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN NaN  ...    NaN   
2019-01-03       NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN NaN  ...    NaN   
2019-01-04       NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN NaN  ...    NaN   
2019-01-07       NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN NaN  ...    NaN   
2019-01-08       NaN  NaN  NaN NaN  NaN  NaN  NaN  NaN  NaN NaN  ...    NaN   

                                                           
           YGMZ  YJ  YQ YSG ZCMD  ZI  ZM ZNTL        ^NDX  
Date                                                       
2019-01-02  NaN NaN NaN NaN  NaN NaN NaN  NaN  2261800000  
2019-01-03  NaN NaN NaN NaN  NaN NaN NaN  NaN  2631550000  
2019-01-04  NaN NaN NaN NaN  NaN NaN NaN  NaN  2596150000  
2019-01

In [51]:
price_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1115 entries, 2019-01-02 to 2023-06-06
Columns: 2232 entries, ('Adj Close', 'ABCL') to ('Volume', '^NDX')
dtypes: float64(2229), int64(3)
memory usage: 19.0 MB


We need to reshape the price data so that we get one row for each date-ticker combination, and columns for the different price types. This way we will get all the ticker symbols as a single column in the df.

In [52]:
price_data_stacked = price_data.stack().reset_index()

In [53]:
print(price_data_stacked.head())

        Date level_1    Adj Close        Close         High          Low  \
0 2019-01-02    THRY    10.350000    10.350000    10.350000    10.350000   
1 2019-01-02    WTRE    22.815611    27.450001    27.450001    27.260000   
2 2019-01-02    ^NDX  6360.870117  6360.870117  6394.740234  6198.680176   
3 2019-01-03    THRY    10.350000    10.350000    10.350000    10.350000   
4 2019-01-03    WTRE    22.848856    27.490000    27.540001    27.430000   

          Open        Volume  
0    10.350000  0.000000e+00  
1    27.280001  7.600000e+03  
2  6198.680176  2.261800e+09  
3    10.350000  0.000000e+00  
4    27.510000  6.300000e+03  


In [54]:
price_data_stacked.rename(columns={'level_1': 'Ticker'}, inplace=True)

In [55]:
print(price_data_stacked.head())

        Date Ticker    Adj Close        Close         High          Low  \
0 2019-01-02   THRY    10.350000    10.350000    10.350000    10.350000   
1 2019-01-02   WTRE    22.815611    27.450001    27.450001    27.260000   
2 2019-01-02   ^NDX  6360.870117  6360.870117  6394.740234  6198.680176   
3 2019-01-03   THRY    10.350000    10.350000    10.350000    10.350000   
4 2019-01-03   WTRE    22.848856    27.490000    27.540001    27.430000   

          Open        Volume  
0    10.350000  0.000000e+00  
1    27.280001  7.600000e+03  
2  6198.680176  2.261800e+09  
3    10.350000  0.000000e+00  
4    27.510000  6.300000e+03  


### Calculating period returns 
This will give us insights into the performance of the companies after they go public.
We will calculate returns over different fixed periods: 1 day, 1 week, 1 month, 3 months, 6 months, and 1 year.

We first group the data by ticker and then use pct_change() method to calculate the percentage change in the 'Close' price for each period. First we will sort the data by 'Date' within each group to ensure the returns are calculated correctly.

In [56]:
price_data_sorted = price_data_stacked.sort_values(by='Date').groupby('Ticker')

price_data_stacked['Return_1d'] = price_data_sorted['Adj Close'].pct_change(1)    # 1-day return
price_data_stacked['Return_1w'] = price_data_sorted['Adj Close'].pct_change(5)    # 1-week return (assuming 5 trading days in a week)
price_data_stacked['Return_1m'] = price_data_sorted['Adj Close'].pct_change(21)   # 1-month return (assuming 21 trading days in a month)
price_data_stacked['Return_3m'] = price_data_sorted['Adj Close'].pct_change(63)   # 3-month return
price_data_stacked['Return_6m'] = price_data_sorted['Adj Close'].pct_change(126)  # 6-month return
price_data_stacked['Return_1y'] = price_data_sorted['Adj Close'].pct_change(252)  # 1-year return


In [57]:
print(price_data_stacked.head(40))

         Date Ticker    Adj Close        Close         High          Low  \
0  2019-01-02   THRY    10.350000    10.350000    10.350000    10.350000   
1  2019-01-02   WTRE    22.815611    27.450001    27.450001    27.260000   
2  2019-01-02   ^NDX  6360.870117  6360.870117  6394.740234  6198.680176   
3  2019-01-03   THRY    10.350000    10.350000    10.350000    10.350000   
4  2019-01-03   WTRE    22.848856    27.490000    27.540001    27.430000   
5  2019-01-03   ^NDX  6147.129883  6147.129883  6292.680176  6139.399902   
6  2019-01-04   SXTC    22.959999    22.959999    23.959999    17.600000   
7  2019-01-04   THRY    10.350000    10.350000    10.350000    10.350000   
8  2019-01-04   WTRE    23.488857    28.260000    28.299999    27.950001   
9  2019-01-04   ^NDX  6422.669922  6422.669922  6445.200195  6234.589844   
10 2019-01-07   SXTC    24.440001    24.440001    27.200001    22.740000   
11 2019-01-07   THRY    10.350000    10.350000    10.350000    10.350000   
12 2019-01-0

In [58]:
# Create a dataframe for NASDAQ 100's return data
nasdaq_return_data = price_data_stacked[price_data_stacked['Ticker'] == '^NDX'][['Date', 'Return_1d', 'Return_1w', 'Return_1m', 'Return_3m', 'Return_6m', 'Return_1y']]


In [59]:
print(nasdaq_return_data)

             Date  Return_1d  Return_1w  Return_1m  Return_3m  Return_6m  \
2      2019-01-02        NaN        NaN        NaN        NaN        NaN   
5      2019-01-03  -0.033602        NaN        NaN        NaN        NaN   
9      2019-01-04   0.044824        NaN        NaN        NaN        NaN   
13     2019-01-07   0.010211        NaN        NaN        NaN        NaN   
18     2019-01-08   0.009802        NaN        NaN        NaN        NaN   
...           ...        ...        ...        ...        ...        ...   
267221 2023-05-31  -0.007029   0.042534   0.077287   0.193953   0.230100   
267542 2023-06-01   0.013149   0.061526   0.101257   0.198976   0.255407   
267862 2023-06-02   0.007280   0.043628   0.116378   0.183538   0.209191   
268182 2023-06-05   0.000678   0.018050   0.121242   0.183217   0.208822   
268502 2023-06-06   0.000109   0.014148   0.097967   0.197983   0.213755   

        Return_1y  
2             NaN  
5             NaN  
9             NaN  
13     

### Merge 
The next step is to merge these two DataFrames to combine the information. However, we want to only keep the stock price data that occurred after the IPO date for each company.

We will keep all columns for now.

In [60]:
merged_data = pd.merge(price_data_stacked, merged_df, left_on='Ticker', right_on='Ticker', how='inner')

In [61]:
print(merged_data.head())

        Date Ticker  Adj Close  Close   High    Low   Open  Volume  Return_1d  \
0 2019-01-02   THRY      10.35  10.35  10.35  10.35  10.35     0.0        NaN   
1 2019-01-03   THRY      10.35  10.35  10.35  10.35  10.35     0.0        0.0   
2 2019-01-04   THRY      10.35  10.35  10.35  10.35  10.35     0.0        0.0   
3 2019-01-07   THRY      10.35  10.35  10.35  10.35  10.35     0.0        0.0   
4 2019-01-08   THRY      10.35  10.35  10.35  10.35  10.35     0.0        0.0   

   Return_1w  ...      CUSIP  Rollup   VC  Dual PostIssueShares Internet  \
0        NaN  ...  886029206       0  0.0   0.0             NaN      0.0   
1        NaN  ...  886029206       0  0.0   0.0             NaN      0.0   
2        NaN  ...  886029206       0  0.0   0.0             NaN      0.0   
3        NaN  ...  886029206       0  0.0   0.0             NaN      0.0   
4        NaN  ...  886029206       0  0.0   0.0             NaN      0.0   

  CRSP permanent ID   Founding  IPO Price  Age at IPO  


In [62]:
merged_data = merged_data[merged_data['Date'] >= merged_data['Offer date']]


In [63]:
print(merged_data.head())

          Date Ticker  Adj Close   Close    High     Low   Open    Volume  \
441 2020-10-01   THRY     11.075  11.075  14.000  10.600  14.00    9600.0   
442 2020-10-02   THRY     14.390  14.390  14.974  12.290  13.44  208900.0   
443 2020-10-05   THRY     14.030  14.030  15.550  13.473  15.10   69700.0   
444 2020-10-06   THRY     12.940  12.940  14.510  12.940  14.45   22600.0   
445 2020-10-07   THRY     11.770  11.770  12.840  11.710  12.84   20600.0   

     Return_1d  Return_1w  ...      CUSIP  Rollup   VC  Dual PostIssueShares  \
441   0.888321   0.888321  ...  886029206       0  0.0   0.0             NaN   
442   0.299323   1.453538  ...  886029206       0  0.0   0.0             NaN   
443  -0.025017   1.392157  ...  886029206       0  0.0   0.0             NaN   
444  -0.077691   1.206309  ...  886029206       0  0.0   0.0             NaN   
445  -0.090417   1.006820  ...  886029206       0  0.0   0.0             NaN   

    Internet CRSP permanent ID   Founding  IPO Price  Ag

In [64]:
print(merged_data.dtypes)

Date                 datetime64[ns]
Ticker                       object
Adj Close                   float64
Close                       float64
High                        float64
Low                         float64
Open                        float64
Volume                      float64
Return_1d                   float64
Return_1w                   float64
Return_1m                   float64
Return_3m                   float64
Return_6m                   float64
Return_1y                   float64
Offer date           datetime64[ns]
IPO name                     object
CUSIP                        object
Rollup                       object
VC                          float64
Dual                        float64
PostIssueShares             float64
Internet                    float64
CRSP permanent ID            object
Founding             datetime64[ns]
IPO Price                   float64
Age at IPO                    int32
dtype: object


In [65]:
print(merged_data.isnull().sum())

Date                      0
Ticker                    0
Adj Close                 0
Close                     0
High                      0
Low                       0
Open                      0
Volume                    0
Return_1d               318
Return_1w              1603
Return_1m              6755
Return_3m             20283
Return_6m             40632
Return_1y             81330
Offer date                0
IPO name                  0
CUSIP                     0
Rollup                    0
VC                        0
Dual                      0
PostIssueShares      175772
Internet                  0
CRSP permanent ID         0
Founding                  0
IPO Price                 0
Age at IPO                0
dtype: int64


In [66]:
print(merged_data.describe())

                                Date      Adj Close          Close  \
count                         266493  266493.000000  266493.000000   
mean   2021-09-26 05:19:19.905888768      32.921112      33.076552   
min              2019-01-04 00:00:00       0.000010       0.000010   
25%              2020-12-22 00:00:00       6.710000       6.830000   
50%              2021-10-15 00:00:00      16.740000      17.049999   
75%              2022-08-10 00:00:00      33.259998      33.369999   
max              2023-06-06 00:00:00    4220.000000    4220.000000   
std                              NaN      80.222517      80.208356   

                High            Low           Open        Volume  \
count  266493.000000  266493.000000  266493.000000  2.664930e+05   
mean       34.267947      32.008334      33.171136  1.455656e+06   
min         0.000010       0.000010       0.000010  0.000000e+00   
25%         7.120000       6.585000       6.860000  6.570000e+04   
50%        17.600000      16.

In [67]:
print(merged_data.columns)

Index(['Date', 'Ticker', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'Return_1d', 'Return_1w', 'Return_1m', 'Return_3m', 'Return_6m',
       'Return_1y', 'Offer date', 'IPO name', 'CUSIP', 'Rollup', 'VC', 'Dual',
       'PostIssueShares', 'Internet', 'CRSP permanent ID', 'Founding',
       'IPO Price', 'Age at IPO'],
      dtype='object')


### Calculating above-average returns
We also want to find the excess return of each IPO over some benchmark, in this case we will use Nasdaq 100 index as benchmark.

We can now calculate the excess returns by subtracting the NASDAQ returns from the IPO returns for each period

In [68]:
# Recall the dataframe for NASDAQ 100's return data

nasdaq_return_data = price_data_stacked[price_data_stacked['Ticker'] == '^NDX'][['Date', 'Return_1d', 'Return_1w', 'Return_1m', 'Return_3m', 'Return_6m', 'Return_1y']]

# Merge with merged_data
merged_data = pd.merge(merged_data, nasdaq_return_data, on='Date', suffixes=('', '_ndx'))

# Calculate excess returns
merged_data['Excess_Return_1d'] = merged_data['Return_1d'] - merged_data['Return_1d_ndx']
merged_data['Excess_Return_1w'] = merged_data['Return_1w'] - merged_data['Return_1w_ndx']
merged_data['Excess_Return_1m'] = merged_data['Return_1m'] - merged_data['Return_1m_ndx']
merged_data['Excess_Return_3m'] = merged_data['Return_3m'] - merged_data['Return_3m_ndx']
merged_data['Excess_Return_6m'] = merged_data['Return_6m'] - merged_data['Return_6m_ndx']
merged_data['Excess_Return_1y'] = merged_data['Return_1y'] - merged_data['Return_1y_ndx']


In [69]:
print(merged_data[['Return_1d', 'Return_1w', 'Return_1m', 'Return_3m', 'Excess_Return_1d', 'Excess_Return_3m']].describe())

           Return_1d      Return_1w      Return_1m      Return_3m  \
count  266175.000000  264890.000000  259738.000000  246210.000000   
mean        0.000718       0.001400      -0.001451      -0.013229   
std         0.164837       0.279309       0.356005       0.572790   
min        -0.972516      -0.986207      -0.987897      -0.990244   
25%        -0.025263      -0.061586      -0.143587      -0.280869   
50%        -0.000940      -0.005714      -0.024512      -0.078349   
75%         0.021978       0.047872       0.093731       0.129888   
max        53.000000      60.818180      57.981814      69.672724   

       Excess_Return_1d  Excess_Return_3m  
count     266175.000000     246210.000000  
mean           0.000045         -0.043853  
std            0.164101          0.556762  
min           -0.986964         -1.148976  
25%           -0.024536         -0.296232  
50%           -0.002301         -0.106394  
75%            0.019865          0.093969  
max           52.993906   

In [70]:
print(merged_data.columns)

Index(['Date', 'Ticker', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'Return_1d', 'Return_1w', 'Return_1m', 'Return_3m', 'Return_6m',
       'Return_1y', 'Offer date', 'IPO name', 'CUSIP', 'Rollup', 'VC', 'Dual',
       'PostIssueShares', 'Internet', 'CRSP permanent ID', 'Founding',
       'IPO Price', 'Age at IPO', 'Return_1d_ndx', 'Return_1w_ndx',
       'Return_1m_ndx', 'Return_3m_ndx', 'Return_6m_ndx', 'Return_1y_ndx',
       'Excess_Return_1d', 'Excess_Return_1w', 'Excess_Return_1m',
       'Excess_Return_3m', 'Excess_Return_6m', 'Excess_Return_1y'],
      dtype='object')


In [71]:
non_nan_counts = merged_data[['Return_1d', 'Return_1m', 'Excess_Return_1d', 'Excess_Return_3m']].count()
print(non_nan_counts)


Return_1d           266175
Return_1m           259738
Excess_Return_1d    266175
Excess_Return_3m    246210
dtype: int64


Manual check if the returns are calculated correctly. 

Important that The "manual" method is calculating the simple return, which is suitable for single-period return calculations.

The method used in the 'Return_1d' column from the DataFrame is calculating the compounded return, which is more appropriate for calculating returns over multiple periods.

In [74]:
date1 = pd.to_datetime('2021-09-30')
date2 = pd.to_datetime('2022-03-23')


In [76]:
# Select rows for a specific ticker and set 'Date' as the index
merged_data_ticker = merged_data[merged_data['Ticker'] == 'ALEC'].set_index('Date')

# Fetch the prices at the two dates
price1 = merged_data_ticker.loc[date1, 'Adj Close']
price2 = merged_data_ticker.loc[date2, 'Adj Close']

# Calculate the return
manual_return = (price2 - price1) / price1
print("Manually calculated return: ", manual_return)

# Check the calculated return in our DataFrame
# Fetch the daily returns for all dates between date1 and date2 (inclusive)
daily_returns = merged_data_ticker.loc[date1:date2, 'Return_1d']

# Calculate the overall return from the daily returns by compounding
calculated_return = (daily_returns + 1).prod() - 1
print("DataFrame calculated return: ", calculated_return)


Manually calculated return:  -0.3597721361937211
DataFrame calculated return:  -0.37776831594922045


The NaN´s in the return and excess-return variables are expected since it is not possible to compute 3-month or 1-year returns until a sufficient amount of time has passed since the IPO.
We will deal with `PostIssueShares` later, but we will probably drop it.

In [80]:
print(merged_data.isnull().sum())

Date                      0
Ticker                    0
Adj Close                 0
Close                     0
High                      0
Low                       0
Open                      0
Volume                    0
Return_1d               318
Return_1w              1603
Return_1m              6755
Return_3m             20283
Return_6m             40632
Return_1y             81330
Offer date                0
IPO name                  0
CUSIP                     0
Rollup                    0
VC                        0
Dual                      0
PostIssueShares      175772
Internet                  0
CRSP permanent ID         0
Founding                  0
IPO Price                 0
Age at IPO                0
Return_1d_ndx             0
Return_1w_ndx             4
Return_1m_ndx            79
Return_3m_ndx           652
Return_6m_ndx          3671
Return_1y_ndx         16645
Excess_Return_1d        318
Excess_Return_1w       1603
Excess_Return_1m       6755
Excess_Return_3m    

### Save DataFrame as a CSV file:

In [79]:
merged_data.to_csv('merged_data.csv')

Below, we have tried to download industry/sector data for our IPOs. Unfortunately the data is not available :(

In [ ]:
# List of tickers
tickers = merged_df['Ticker'].unique().tolist()

# Initialize a dataframe to store the data
data = pd.DataFrame(columns=["Ticker", "Industry"])

for ticker in tickers:
    try:
        info = yf.Ticker(ticker).info
        industry = info['industry']
        data = data.append({"Ticker": ticker, "Industry": industry}, ignore_index=True)
    except:
        pass

print(data)